# Module 7: Modern Tooling and Packaging

### The Scenario

You've written a great Python application. Now you need to:
- Install dependencies without breaking your system Python
- Share your project so others can run it
- Set up a reproducible development environment

### The Goal

By the end of this module, you will:
- Understand **pip**, **conda**, and **uv** - when to use each
- Create and manage **virtual environments**
- Build a **professional development workflow**

---

## Lesson 1: Package Managers - The Evolution

### The Problem

You run `pip install numpy` and it breaks another project that needed an older version. Or worse, it breaks your system tools.

### The "Aha!" Moment

Package managers have evolved to solve isolation and reproducibility:

| Era | Tool | Key Innovation |
|-----|------|---------------|
| 2008 | pip | Standard package installer |
| 2012 | conda | Environment + non-Python packages |
| 2024 | uv | Rust-based, 10-100x faster |

### pip - The Standard

**pip** (Pip Installs Packages) is Python's default package manager.

| Command | Purpose |
|---------|--------|
| `pip install package` | Install latest version |
| `pip install package==1.2.3` | Install specific version |
| `pip install -r requirements.txt` | Install from file |
| `pip install -e .` | Editable install (development) |
| `pip freeze > requirements.txt` | Export installed packages |
| `pip list` | Show installed packages |
| `pip show package` | Package details |

In [ ]:
# Check pip version and location
!pip --version

In [ ]:
# Show installed packages
!pip list | head -10

In [ ]:
# Show package info
!pip show pip

### requirements.txt Format

```
# Exact versions (reproducible)
requests==2.31.0
numpy==1.26.0

# Minimum version
pandas>=2.0.0

# Version range
flask>=2.0,<3.0

# From git
git+https://github.com/user/repo.git@v1.0

# Local package
-e ./my_local_package
```

### conda - Environment + Packages

**conda** handles both Python packages AND non-Python dependencies (C libraries, CUDA, etc.).

| Feature | pip | conda |
|---------|-----|-------|
| Package source | PyPI | Anaconda/conda-forge |
| Non-Python deps | No | Yes (CUDA, MKL, etc.) |
| Environment mgmt | Separate (venv) | Built-in |
| Dependency solver | Basic | SAT solver |
| Best for | Pure Python | Data science, ML |

### conda Commands

| Command | Purpose |
|---------|--------|
| `conda create -n myenv python=3.12` | Create environment |
| `conda activate myenv` | Activate environment |
| `conda install numpy` | Install package |
| `conda install -c conda-forge package` | Install from channel |
| `conda env export > environment.yml` | Export environment |
| `conda env create -f environment.yml` | Create from file |
| `conda list` | Show installed packages |

### environment.yml Format

```yaml
name: myproject
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.12
  - numpy=1.26
  - pandas>=2.0
  - cudatoolkit=11.8  # Non-Python!
  - pip:
    - some-pypi-only-package
```

---

## Lesson 2: uv - The Future

### The Problem

`pip install` takes forever. Creating virtual environments is slow. Dependency resolution fails mysteriously.

### The "Aha!" Moment

**uv** is a Rust-based package manager that's 10-100x faster than pip. It's a drop-in replacement that also handles virtual environments.

### uv vs pip Performance

| Operation | pip | uv |
|-----------|-----|----|
| Install numpy | ~5s | ~0.1s |
| Create venv | ~3s | ~0.01s |
| Resolve dependencies | Slow | Fast (parallel) |
| Lock file | No | Yes |

### Installing uv

```bash
# macOS/Linux
curl -LsSf https://astral.sh/uv/install.sh | sh

# or with pip
pip install uv

# or with Homebrew
brew install uv
```

### uv Commands

| Command | Purpose |
|---------|--------|
| `uv venv` | Create virtual environment |
| `uv pip install package` | Install package |
| `uv pip compile requirements.in` | Generate locked requirements |
| `uv pip sync requirements.txt` | Sync environment to requirements |
| `uv run python script.py` | Run in managed environment |
| `uv init` | Initialize new project |
| `uv add package` | Add dependency to project |

In [ ]:
# Check if uv is installed
!which uv || echo "uv not installed - install with: curl -LsSf https://astral.sh/uv/install.sh | sh"

### uv Project Workflow

```bash
# Initialize new project
uv init myproject
cd myproject

# Add dependencies
uv add requests pandas

# Add dev dependencies
uv add --dev pytest black

# Run your code
uv run python main.py

# Run tests
uv run pytest
```

uv creates a `pyproject.toml` and `uv.lock` file automatically!

---

## Lesson 3: Virtual Environments

### The Problem

Project A needs `numpy==1.20`. Project B needs `numpy==1.26`. How do you work on both?

### The "Aha!" Moment

Virtual environments create **isolated Python installations**. Each project gets its own packages.

### Virtual Environment Options

| Tool | Command | Speed | Notes |
|------|---------|-------|-------|
| venv (built-in) | `python -m venv .venv` | Slow | Standard library |
| virtualenv | `virtualenv .venv` | Medium | More features |
| conda | `conda create -n myenv` | Medium | Includes non-Python |
| uv | `uv venv` | Fast | 10-100x faster |

### Creating and Using Virtual Environments

```bash
# Create with venv (built-in)
python -m venv .venv

# Create with uv (faster)
uv venv

# Activate (macOS/Linux)
source .venv/bin/activate

# Activate (Windows)
.venv\Scripts\activate

# Deactivate
deactivate

# Delete (just remove the folder)
rm -rf .venv
```

In [ ]:
import sys

# Check if we're in a virtual environment
print(f"Python executable: {sys.executable}")
print(f"In virtual env: {sys.prefix != sys.base_prefix}")

### Project Structure with Virtual Environment

```
myproject/
├── .venv/                 # Virtual environment (don't commit!)
├── .gitignore             # Ignore .venv/
├── pyproject.toml         # Project config
├── requirements.txt       # Or use pyproject.toml
├── src/
│   └── mypackage/
└── tests/
```

### .gitignore for Python Projects

```gitignore
# Virtual environments
.venv/
venv/
env/

# Python cache
__pycache__/
*.pyc
*.pyo

# Distribution
dist/
build/
*.egg-info/

# IDE
.idea/
.vscode/

# Environment files with secrets
.env
```

---

## Lesson 4: Professional Development Workflow

### The Problem

Every developer on your team has a slightly different setup. "Works on my machine" becomes a daily problem.

### The "Aha!" Moment

A reproducible workflow ensures everyone has the same environment.

### Recommended Workflow (2024+)

```bash
# 1. Clone project
git clone https://github.com/org/project
cd project

# 2. Create virtual environment
uv venv  # or: python -m venv .venv

# 3. Activate
source .venv/bin/activate

# 4. Install dependencies
uv pip install -e ".[dev]"  # or: pip install -e ".[dev]"

# 5. Run tests
pytest

# 6. Start coding!
```

### Choosing the Right Tool

| Situation | Recommended |
|-----------|-------------|
| New project, pure Python | **uv** |
| Data science, need CUDA/MKL | **conda** |
| Legacy project | **pip + venv** |
| CI/CD, Docker | **pip** (most compatible) |
| Maximum speed | **uv** |

### Dependency Locking

Lock files ensure **exact** versions are installed:

| Tool | Lock File | Command |
|------|-----------|--------|
| pip-tools | `requirements.txt` | `pip-compile` |
| Poetry | `poetry.lock` | `poetry lock` |
| uv | `uv.lock` | Automatic |
| conda | `conda-lock.yml` | `conda-lock` |

### pip-tools Workflow

```bash
# Install pip-tools
pip install pip-tools

# Create requirements.in (loose versions)
echo "requests>=2.28" > requirements.in
echo "pandas>=2.0" >> requirements.in

# Compile to requirements.txt (locked versions)
pip-compile requirements.in

# Install exactly what's in requirements.txt
pip-sync requirements.txt
```

---

## Lesson 5: Docker for Python

### The Problem

Virtual environments don't capture the OS, system libraries, or Python version itself.

### The "Aha!" Moment

Docker containers include **everything** - OS, Python, libraries. True reproducibility.

### Basic Python Dockerfile

```dockerfile
# Use official Python image
FROM python:3.12-slim

# Set working directory
WORKDIR /app

# Install dependencies first (caching)
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Run the application
CMD ["python", "main.py"]
```

### Optimized Dockerfile with uv

```dockerfile
FROM python:3.12-slim

# Install uv
COPY --from=ghcr.io/astral-sh/uv:latest /uv /usr/local/bin/uv

WORKDIR /app

# Install dependencies (cached layer)
COPY pyproject.toml uv.lock ./
RUN uv sync --frozen --no-dev

# Copy application
COPY . .

CMD ["uv", "run", "python", "main.py"]
```

---

## Summary

### Package Manager Comparison

| Feature | pip | conda | uv |
|---------|-----|-------|----|
| Speed | Slow | Medium | Fast |
| Non-Python deps | No | Yes | No |
| Lock files | No* | No* | Yes |
| Venv management | No | Yes | Yes |
| Best for | Compatibility | Data science | Speed |

*Use pip-tools or conda-lock

### Quick Reference

| Task | pip | conda | uv |
|------|-----|-------|----|
| Install | `pip install pkg` | `conda install pkg` | `uv pip install pkg` |
| Create env | `python -m venv .venv` | `conda create -n env` | `uv venv` |
| Activate | `source .venv/bin/activate` | `conda activate env` | `source .venv/bin/activate` |
| Export | `pip freeze` | `conda env export` | `uv pip compile` |

### Best Practices

| Practice | Why |
|----------|-----|
| Always use virtual environments | Isolation |
| Pin dependency versions | Reproducibility |
| Use lock files | Exact versions |
| Don't commit `.venv/` | Size, platform-specific |
| Separate dev dependencies | Smaller production installs |

---

**Course Complete!** You now have a solid foundation in Python internals, data structures, concurrency, and modern tooling.